# basic idea
- 기존 ML 기반 클러스터링은 잘 작동하지 않았음
- 텍스트가 주제 분류될 때 복잡한 과정을 거치기 때문 ex) KT wiz와 삼성 라이온즈의 맞대결 승자는? &rarr; 스포츠; 하지만 KT와 삼성이라는 단어 때문에 경제로 분류될 가능성 있음.
- 복잡한 로직의 처리를 LLM으로 하면 어떨까? &rarr; LLM 사용 시 출력 컨트롤이 관건
    1. 출력이 영어로 되는 경우 - 프롬프트에서 한국어 명시
    2. 출력이 균일하게 되지 않는 경우 - seed와 temperature 관리
    3. 출력에 노이즈가 생기는 경우 - 노이즈 양에 따라 허용 or 무시

In [8]:
import torch
import re

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel

In [2]:
# 14번 이슈 PR merge 시 코드 바꿀 것
import re
import pandas as pd

df: pd.DataFrame = pd.read_csv('data/train.csv')

df['text'] = df['text'].str.replace('…', ' ', regex=False)
df['text'] = df['text'].str.replace('...', ' ', regex=False)
df['text'] = df['text'].str.replace('·', ' ', regex=False)

df['text'] = df['text'].str.replace('→', '에서', regex=False)
df['text'] = df['text'].str.replace('↑', '상승', regex=False)
df['text'] = df['text'].str.replace('↓', '하락', regex=False)
df['text'] = df['text'].str.replace('↔', ' ', regex=False)

# 특수 문자 감지하는 regex
SPECIAL_CHAR_PATTERN = r'(?<!\d)\.(?!\d)|(?<!\d)%|[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]'

# 실험적으로 알아낸 비율 값
SPECIAL_CHAR_RATIO_THRESHOLD = .042

def count_special_characters(text):
    return len(re.findall(SPECIAL_CHAR_PATTERN, text))

df['special_char_count'] = df['text'].apply(count_special_characters)
df['special_char_ratio'] = df['special_char_count'] / df['text'].str.len()

noise_df = df[df['special_char_ratio'] >= SPECIAL_CHAR_RATIO_THRESHOLD]
clean_df = df[~df.index.isin(noise_df.index)]
print(len(noise_df), len(clean_df))
noise_df.head()

1595 1205


,ID,text,target,special_char_count,special_char_ratio
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,6,0.187500
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,7,0.259259
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,5,0.227273
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,5,0.178571
6,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,3,0.142857


In [3]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            """
            You are a helpful assistant that categories news article titles to propre sections. 
            only say in a short korean word.
            """,
        ),
        (
            "human",
            "{input}",
        )
    ]
)

llm = ChatOllama(
    model="gemma2:27b",
    seed=42,
    temperature=0
)

chain = prompt | llm

In [4]:
clean_df.loc[:, 'predict_label'] = ""

for idx, row in clean_df.iterrows():
    input_text = row['text']
    ai_msg = chain.invoke({"input": input_text})
    predicted_label = ai_msg.content.strip()  # 결과 문자열에서 공백 제거
    clean_df.loc[idx, 'predict_label'] = predicted_label
    print(f"Index: {idx}, Input: {input_text}, Predicted Label: {predicted_label}")

/var/folders/f9/7w3cw7_s6c7849zf1svk37000000gn/T/ipykernel_44869/3744849456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.loc[:, 'predict_label'] = ""


Index: 3, Input: 갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩, Predicted Label: 경제
Index: 5, Input: 美성인 6명 중 1명꼴 배우자 연인 빚 떠안은 적 있다, Predicted Label: 사회
Index: 7, Input: 아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳 , Predicted Label: 농구 🏀
Index: 8, Input: 朴대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문종합, Predicted Label: 사회
Index: 9, Input: 듀얼심 아이폰 하반기 출시설 솔솔 알뜰폰 기대감, Predicted Label: 전자제품
Index: 11, Input: NH투자 1월 옵션 만기일 매도 우세, Predicted Label: 금융
Index: 12, Input: 황총리 각 부처 비상대비태세 철저히 강구해야, Predicted Label: 정치
Index: 15, Input: 게시판 KISA 박민정 책임연구원 APTLD 이사 선출, Predicted Label: 기술
Index: 16, Input: 공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예, Predicted Label: 부패
Index: 17, Input: 월세 전환에 늘어나는 주거비 부담 작년 역대 최고치, Predicted Label: 부동산
Index: 19, Input: 페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공, Predicted Label: 과학기술
Index: 20, Input: 추신수 타율 0.265로 시즌 마감 최지만은 19홈런 6 , Predicted Label: 스포츠 ⚾
Index: 23, Input: 아시안게임 목소리 높인 박항서 베트남이 일본 못 이길 , Predicted Label: 스포츠
Index: 24, Input: 서울에 다시 오존주의보 도심 서북 동북권 발령종합, Predicted Label: 환경
Index: 34, Input: 안보리 대북결의안 2270호 이행보고서 

위 실행 결과를 보면, 다양한 label로 분류되는 걸 알 수 있다. 따라서 label 텍스트 자체를 임베딩해 클러스터링을 시도한다.

In [5]:
def clean_predict_label(text):
    return re.sub(r'[^가-힣a-zA-Z\s]', '', text)

clean_df.loc[:, 'predict_label'] = clean_df['predict_label'].apply(clean_predict_label)

In [6]:
label_counts = clean_df['predict_label'].value_counts()
label_counts

predict_label
경제      189
정치      158
스포츠      71
기술       62
사회       61
       ... 
치안        1
체육        1
여행휴식      1
젊음        1
IT기술      1
Name: count, Length: 141, dtype: int64

In [9]:
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [10]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32, padding="max_length")
    outputs = model(**inputs)
    # [CLS] 토큰의 출력만 사용하여 임베딩 생성
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().numpy()
    return embedding

In [13]:
embeddings = clean_df['predict_label'].apply(get_embedding).tolist()
embeddings = torch.tensor(embeddings)  # 리스트를 텐서로 변환

# KMeans 클러스터링 (7개의 클러스터로 설정)
kmeans = KMeans(n_clusters=7, random_state=42)
clean_df['label_cluster'] = kmeans.fit_predict(embeddings)

# 클러스터링 결과 확인
print(clean_df[['predict_label', 'label_cluster']].head())

  predict_label  label_cluster
3            경제              4
5            사회              2
7           농구               3
8            사회              2
9          전자제품              6


/var/folders/f9/7w3cw7_s6c7849zf1svk37000000gn/T/ipykernel_44869/2511216210.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['label_cluster'] = kmeans.fit_predict(embeddings)


In [14]:
cluster_counts = clean_df['label_cluster'].value_counts()
print("\nCluster distribution:")
print(cluster_counts)


Cluster distribution:
label_cluster
4    329
0    277
1    265
3    155
2    106
6     54
5     19
Name: count, dtype: int64


In [16]:
cross_tab = pd.crosstab(clean_df['label_cluster'], clean_df['target'])
cross_tab

target,0,1,2,3,4,5,6
label_cluster,,,,,,,
0,36,40,36,40,36,49,40
1,41,39,48,29,37,31,40
2,19,18,11,10,17,19,12
3,23,29,11,27,27,20,18
4,43,45,43,42,53,55,48
5,2,5,1,8,1,1,1
6,6,7,10,3,8,14,6
